Import the desired packages and libraries

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

Load the .csv file and save it into the dataframe

In [ ]:
#read the CSV file
tpot_data = pd.read_csv('C:\\MSIS Sem 2\\DSMT - Assignment 3\\featuredEngineering1.csv', dtype=np.float64)
tpot_data.shape

The interest rate is labeled as "target" and based on the features decided in part 2, we will exclude the remaining features which are not useful and create a final dataset.

In [ ]:
#drop the columns not in use
tpot_data_new = tpot_data.drop(['member_id', 'emp_length',  'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal',  'total_acc', 'last_pymnt_amnt', 'collections_12_mths_ex_med', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim', 'home_ownership_ANY', 'home_ownership_MORTGAGE', 'home_ownership_NONE', 'home_ownership_OTHER', 'home_ownership_OWN', 'home_ownership_RENT', 'verification_status_Not Verified', 'verification_status_Source Verified', 'purpose_car', 'purpose_credit_card', 'purpose_educational', 'purpose_home_improvement', 'purpose_house', 'purpose_major_purchase', 'purpose_medical', 'purpose_moving', 'purpose_other', 'purpose_renewable_energy', 'purpose_small_business', 'purpose_vacation', 'purpose_wedding', 'term_ 36 months',  'acc_ratio', 'target','annual_inc'], axis=1)

Now we will split the data set in train and test, create and fit a TPOT regressor. TPOT will display the "best" model hyperparameters

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor

tpot_ir = tpot_data['target'].values
X_train, X_test, y_train, y_test = train_test_split(tpot_data_new, tpot_ir, train_size=0.75, test_size=0.25)
tpot = TPOTRegressor(generations=5, population_size=20, verbosity=2)
tpot.fit(X_train, y_train)

In [ ]:
print("TPOT cross-validation MSE")
print(tpot.score(X_test, y_test))

In [ ]:
from sklearn.metrics import mean_squared_error
print('MSE:')
print(mean_squared_error(y_test, tpot.predict(X_test)))

The difference between our predicted interest rate and the actual one

In [ ]:
print('RMSE:')
print(np.sqrt(mean_squared_error(y_test, tpot.predict(X_test))))

Output the pipeline as an execution-ready Python script file for a later use

In [ ]:
tpot.export('tpot_lendingClub_interest_rate_pipeline.py')

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator

# NOTE: Make sure that the class is labeled 'target' in the data file
tpot_data_f = pd.read_csv('C:\\MSIS Sem 2\\DSMT - Assignment 3\\featuredEngineering2-small.csv', dtype=np.float64)
features = tpot_data_f.drop(['member_id', 'emp_length',  'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal',  'total_acc', 'last_pymnt_amnt', 'collections_12_mths_ex_med', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim', 'home_ownership_ANY', 'home_ownership_MORTGAGE', 'home_ownership_NONE', 'home_ownership_OTHER', 'home_ownership_OWN', 'home_ownership_RENT', 'verification_status_Not Verified', 'verification_status_Source Verified', 'purpose_car', 'purpose_credit_card', 'purpose_educational', 'purpose_home_improvement', 'purpose_house', 'purpose_major_purchase', 'purpose_medical', 'purpose_moving', 'purpose_other', 'purpose_renewable_energy', 'purpose_small_business', 'purpose_vacation', 'purpose_wedding', 'term_ 36 months',  'acc_ratio', 'target','annual_inc'], axis=1).values
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data_f['target'].values, random_state=None)

# Average CV score on the training set was:-74.90881962449828
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=KNeighborsRegressor(n_neighbors=47, p=1, weights="uniform")),
    RandomForestRegressor(bootstrap=True, max_features=0.25, min_samples_leaf=16, min_samples_split=4, n_estimators=100)
)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)
results